In [1]:
import numpy as np,pandas as pd,json,sqlite3,requests, os
from jsonpath_ng import jsonpath
from jsonpath_ng.ext import parse
from pandas.io.json import json_normalize
from dotenv import load_dotenv
load_dotenv()
db_path = os.environ.get('dsa_db')
char_path = os.environ.get('char_data')
game_path = os.environ.get('game_data')
api_kiki = os.environ.get('api_kiki')
url_gm = os.environ.get('url_game_data')
url_chars = os.environ.get('url_chars')
headers = os.environ.get('headers')

In [3]:
with open(game_path,'r') as g_data:
    j_data = json.load(g_data)

In [6]:
data_path = j_data['gear_slot_tables']['3']['Data']

In [15]:
c = pd.json_normalize(data_path)
b = list(c.columns)
stats = []
for i in b:
    i = i.replace('.Tiers','')
    stats.append(i)

In [25]:
stats

['Offense',
 'Defense',
 'Health',
 'Recovery',
 'CritChance',
 'CritPower',
 'AttackSpeed',
 'Speed',
 'Potency',
 'Tenacity',
 'Accuracy',
 'Evasion']

In [85]:
demo = j_data['gear_slot_tables']['3']['Data'][stats[0]]['Tiers']

In [86]:
demo

[{'Values': [10.0, 10.0, 10.0, 10.0]},
 {'Values': [20.0, 20.0, 20.0, 20.0]},
 {'Values': [30.0, 30.0, 30.0, 30.0]},
 {'Values': [40.0, 40.0, 40.0, 40.0]},
 {'Values': [50.0, 50.0, 50.0, 50.0]},
 {'Values': [60.0, 60.0, 60.0, 60.0]},
 {'Values': [70.0, 70.0, 70.0, 105.0]},
 {'Values': [80.0, 80.0, 80.0, 120.0]},
 {'Values': [90.0, 90.0, 90.0, 135.0]},
 {'Values': [100.0, 100.0, 100.0, 150.0]},
 {'Values': [110.0, 110.0, 110.0, 165.0]},
 {'Values': [120.0, 120.0, 120.0, 180.0]}]

In [138]:
def fix_gear(dataf):
    g = 1
    c = 1
    length = dataf.shape[0]
    for i in range(0,length):
        if i % 4 == 0 and i != 0: g+=1
        dataf.at[i,'Gear'] = g
        dataf.at[i,'Item_order'] = c
        c+=1
        if c == 5: c = 1 
    return dataf

In [139]:
dframe = pd.json_normalize(demo,record_path=['Values'],errors='ignore')
dframe =  dframe[:32]
dframe = fix_gear(dframe)


In [140]:
dframe.drop(dframe['Item_order'==2].index)

,0,Gear,Item_order


In [141]:
dframe

,0,Gear,Item_order
0,10.0,1.0,1.0
1,10.0,1.0,2.0
2,10.0,1.0,3.0
3,10.0,1.0,4.0
4,20.0,2.0,1.0
5,20.0,2.0,2.0
6,20.0,2.0,3.0
7,20.0,2.0,4.0
8,30.0,3.0,1.0
9,30.0,3.0,2.0


In [142]:
check = dframe[dframe['Item_order'].isin([1,4])]
type(check)

pandas.core.frame.DataFrame

In [143]:
check = check.rename(columns={0:'Addition'})

In [144]:
check['Stat'] = stats[0]

In [131]:
type(check)

pandas.core.frame.DataFrame

In [118]:
check.columns

Index(['Addition', 'Gear', 'Item_order', 'Stat'], dtype='object')

In [145]:
check

,Addition,Gear,Item_order,Stat
0,10.0,1.0,1.0,Offense
3,10.0,1.0,4.0,Offense
4,20.0,2.0,1.0,Offense
7,20.0,2.0,4.0,Offense
8,30.0,3.0,1.0,Offense
11,30.0,3.0,4.0,Offense
12,40.0,4.0,1.0,Offense
15,40.0,4.0,4.0,Offense
16,50.0,5.0,1.0,Offense
19,50.0,5.0,4.0,Offense


In [146]:
check = check.loc[,['Stat','Gear','Addition','Item_order']]

SyntaxError: invalid syntax (2811436828.py, line 1)

In [150]:
with sqlite3.connect(db_path) as conn:
    conn.execute('DROP TABLE IF EXISTS item_upgrades_gear')
    for stat in stats:
        data_path = j_data['gear_slot_tables']['3']['Data'][stat]['Tiers']
        dframe = pd.json_normalize(data_path,record_path=['Values'],errors='ignore')
        dframe =  dframe[:32]
        dframe = fix_gear(dframe)
        a_data = dframe[dframe['Item_order'].isin([1,4])]
        a_data = a_data.rename(columns={0:'Addition'})
        a_data['Stat'] = stat
        a_data = a_data[['Stat','Gear','Addition','Item_order']]
        a_data.to_sql('item_upgrades_gear', conn, if_exists='append',index=False)